#Importing Libraries

In [ ]:
import keras
import tensorflow
import theano
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

#Import dataset


In [ ]:
dataset = pd.read_csv("Churn_Modelling.csv")
X = dataset.iloc[:, 3 : 13].values
y = dataset.iloc[:, -1].values

In [ ]:
X

array([[619, 'France', 'Female', ..., 1, 1, 101348.88],
       [608, 'Spain', 'Female', ..., 0, 1, 112542.58],
       [502, 'France', 'Female', ..., 1, 0, 113931.57],
       ...,
       [709, 'France', 'Female', ..., 0, 1, 42085.58],
       [772, 'Germany', 'Male', ..., 1, 0, 92888.52],
       [792, 'France', 'Female', ..., 1, 0, 38190.78]], dtype=object)

In [ ]:
X.shape

(10000, 10)

In [ ]:
y

array([1, 0, 1, ..., 1, 1, 0])

#Encoding categorical data


In [ ]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
ct = ColumnTransformer(transformers = [('encoder', OneHotEncoder(categories = 'auto'), [1])], remainder ='passthrough')
X = np.array(ct.fit_transform(X))
labelencoder_X_2 = LabelEncoder()
X[:, 4] = labelencoder_X_2.fit_transform(X[:, 4])

In [ ]:
X

array([[1.0, 0.0, 0.0, ..., 1, 1, 101348.88],
       [0.0, 0.0, 1.0, ..., 0, 1, 112542.58],
       [1.0, 0.0, 0.0, ..., 1, 0, 113931.57],
       ...,
       [1.0, 0.0, 0.0, ..., 0, 1, 42085.58],
       [0.0, 1.0, 0.0, ..., 1, 0, 92888.52],
       [1.0, 0.0, 0.0, ..., 1, 0, 38190.78]], dtype=object)

In [ ]:
print(X[1, :])

[0.0 0.0 1.0 608 0 41 1 83807.86 1 0 1 112542.58]


In [ ]:
print(X.shape)

(10000, 12)


In [ ]:
X = X[:, 1:]

In [ ]:
X.shape

(10000, 11)

#Train Test split


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

#Building ANN


In [ ]:
import keras

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

In [ ]:
classifier = Sequential()

In [ ]:
classifier.add(keras.Input(shape = (11, )))
classifier.add(Dense(6, kernel_initializer="uniform", activation = "relu"))

In [ ]:
classifier.add(Dense(6, activation = "relu"))

In [ ]:
classifier.add(Dense(1, kernel_initializer="uniform", activation = "sigmoid"))

In [ ]:
classifier.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])

In [ ]:
classifier.fit(X_train, y_train, batch_size = 10, epochs = 100)

Epoch 1/100
800/800 [==============================] - 1s 1ms/step - loss: 0.4882 - accuracy: 0.7951
Epoch 2/100
800/800 [==============================] - 1s 1ms/step - loss: 0.4198 - accuracy: 0.8199
Epoch 3/100
800/800 [==============================] - 1s 1ms/step - loss: 0.4112 - accuracy: 0.8273
Epoch 4/100
800/800 [==============================] - 1s 1ms/step - loss: 0.3964 - accuracy: 0.8347
Epoch 5/100
800/800 [==============================] - 1s 2ms/step - loss: 0.3666 - accuracy: 0.8516
Epoch 6/100
800/800 [==============================] - 2s 2ms/step - loss: 0.3520 - accuracy: 0.8593
Epoch 7/100
800/800 [==============================] - 2s 2ms/step - loss: 0.3483 - accuracy: 0.8593
Epoch 8/100
800/800 [==============================] - 2s 2ms/step - loss: 0.3461 - accuracy: 0.8614
Epoch 9/100
800/800 [==============================] - 1s 2ms/step - loss: 0.3446 - accuracy: 0.8594
Epoch 10/100
800/800 [==============================] - 1s 2ms/step - loss: 0.3439 - accura

In [ ]:
y_pred = classifier.predict(X_test)

In [ ]:
y_pred

array([[0.19487923],
       [0.32446843],
       [0.08522907],
       ...,
       [0.13514823],
       [0.13704032],
       [0.24381876]], dtype=float32)

In [ ]:
y_pred = (y_pred > 0.5)

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[1525,   70],
       [ 203,  202]])

In [ ]:
from sklearn.metrics import accuracy_score
score = accuracy_score(y_test, y_pred)
score

0.8635

In [ ]:
new_predicton = classifier.predict(sc.transform(np.array([[0.0, 0.0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])))

In [ ]:
new_predicton = new_predicton > 0.5

In [ ]:
new_predicton

array([[False]])

In [ ]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score

In [ ]:
def build_classifier():
  classifier = Sequential()
  classifier.add(keras.Input(shape = (11, )))
  classifier.add(Dense(6, kernel_initializer="uniform", activation = "relu"))
  classifier.add(Dense(6, activation = "relu"))
  classifier.add(Dense(1, kernel_initializer="uniform", activation = "sigmoid"))
  classifier.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])
  return classifier

In [ ]:
classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 100)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = -1)

In [ ]:
mean = accuracies.mean()

In [ ]:
mean

0.8596250057220459

In [ ]:
variances = accuracies.std()
variances

0.010518584663885873

In [ ]:
from keras.layers import Dropout

classifier = Sequential()
classifier.add(keras.Input(shape = (11, )))
classifier.add(Dense(6, kernel_initializer="uniform", activation = "relu"))
classifier.add(Dropout(rate = 0.1))
classifier.add(Dense(6, activation = "relu"))
classifier.add(Dropout(rate = 0.1 ))
classifier.add(Dense(1, kernel_initializer="uniform", activation = "sigmoid"))
classifier.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])

TypeError: ignored

#Tuning the ANN

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
def build_classifier(optimizer):
  classifier = Sequential()
  classifier.add(keras.Input(shape = (11, )))
  classifier.add(Dense(6, kernel_initializer="uniform", activation = "relu"))
  classifier.add(Dense(6, activation = "relu"))
  classifier.add(Dense(1, kernel_initializer="uniform", activation = "sigmoid"))
  classifier.compile(optimizer = optimizer, loss = "binary_crossentropy", metrics = ["accuracy"])
  return classifier
classifier = KerasClassifier(build_fn = build_classifier)
parameters = { 'batch_size' : [25, 32], 
               'epochs' : [100, 500], 
               'optimizer' : ['adam', 'rmsprop']}
gridsearch = GridSearchCV(estimator = classifier, 
                          param_grid = parameters, 
                          scoring = 'accuracy', 
                          cv = 10) 
gridsearch =gridsearch.fit(X_train, y_train)
best_parameters = gridsearch.best_parameters
best_accuracy = gridsearch.best_accuracy

Streaming output truncated to the last 5000 lines.
Epoch 333/500
288/288 [==============================] - 0s 988us/step - loss: 0.3340 - accuracy: 0.8661
Epoch 334/500
288/288 [==============================] - 0s 991us/step - loss: 0.3342 - accuracy: 0.8661
Epoch 335/500
288/288 [==============================] - 0s 1ms/step - loss: 0.3340 - accuracy: 0.8656
Epoch 336/500
288/288 [==============================] - 0s 1ms/step - loss: 0.3344 - accuracy: 0.8665
Epoch 337/500
288/288 [==============================] - 0s 972us/step - loss: 0.3341 - accuracy: 0.8664
Epoch 338/500
288/288 [==============================] - 0s 991us/step - loss: 0.3342 - accuracy: 0.8660
Epoch 339/500
288/288 [==============================] - 0s 1ms/step - loss: 0.3342 - accuracy: 0.8675
Epoch 340/500
288/288 [==============================] - 0s 1ms/step - loss: 0.3343 - accuracy: 0.8647
Epoch 341/500
288/288 [==============================] - 0s 1ms/step - loss: 0.3343 - accuracy: 0.8678
Epoch 342/500
